공공데이터포털(http://data.go.kr/) 동물보호관리시스템 유기동물 조회 서비스에서 유기동물 정보를 조회합니다.
RESTful API 형식으로 제공되며, 축종, 시군구, 상태, 출생년도, 체중, 보호소 등의 정보를 제공합니다.

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import Series,DataFrame
import re
from collections import Counter
import json
from datetime import datetime

In [10]:
url = "http://openapi.animal.go.kr/openapi/service/rest/abandonmentPublicSrvc/abandonmentPublic?serviceKey="
key = "Your_Auth_Key"
param = "&bgnde=20180101&endde=20180131&numOfRows=100000000&pageNo=1&upkind=417000"

데이터를 받아올 url, 인증키(공공데이터포털에서 개별 신청), parameter 값을 지정합니다.
여기서 bgnde는 검색시작일, endde는 검색종료일(유기날짜 기준), numOfRows는 불러올 행의 수, upkind는 축종코드를 의미합니다.
(축종코드 개 417000 고양이 422400)
동물보호관리시스템_OpenAPI활용가이드에서 더 정확한 parameter 관련 정보를 확인가능합니다.

In [11]:
responses = requests.get(url+key+param)
soup = BeautifulSoup(responses.content, 'lxml-xml')

param_list = ['age', 'careAddr', 'careNm', 'careTel', 'chargeNm', 'colorCd', 'desertionNo', 'filename', 
'happenDt', 'happenPlace', 'kindCd', 'neuterYn', 'noticeEdt', 'noticeNo', 'noticeSdt', 'officetel', 
'orgNm', 'popfile', 'processState', 'sexCd', 'specialMark', 'weight']

for i, param in enumerate(param_list) :
    exec( param + ' = []')

for item in soup.findAll('item'):
    for param in param_list :
        eval(param).append(item.find(param).string if item.find(param) else None)

abd_data = pd.DataFrame(dict(zip(param_list, list(map(lambda x : eval(x),param_list)))))

In [12]:
abd_data

,age,careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,happenPlace,...,noticeEdt,noticeNo,noticeSdt,officetel,orgNm,popfile,processState,sexCd,specialMark,weight
0,2015(년생),경상남도 밀양시 부북면 감천리 615-4,밀양시동물보호소(구),010-9185-1919,밀양시,흰색,448536201800086,http://www.animal.go.kr/files/shelter/2018/02/...,20180131,밀양시 부북면 덕곡리110-4,...,20180219,경남-밀양-2018-00098,20180209,055-359-7186,경상남도 밀양시,http://www.animal.go.kr/files/shelter/2018/02/...,종료(입양),M,피부병이 조금 있고 다리가 짧은편이었음,3(Kg)
1,2018(년생),경상남도 밀양시 부북면 감천리 615-4,밀양시동물보호소(구),010-9185-1919,밀양시,흰색,448536201800085,http://www.animal.go.kr/files/shelter/2018/02/...,20180131,밀양시 부북면 덕곡리110-4,...,20180219,경남-밀양-2018-00097,20180209,055-359-7186,경상남도 밀양시,http://www.animal.go.kr/files/shelter/2018/02/...,종료(입양),F,털이 길어 엉켜서 엉망이었으며 경계심이 심했음,4(Kg)
2,2014(년생),경상남도 밀양시 부북면 감천리 615-4,밀양시동물보호소(구),010-9185-1919,밀양시,연갈색,448536201800084,http://www.animal.go.kr/files/shelter/2018/02/...,20180131,밀양시 삼랑진읍 송지리9-5,...,20180219,경남-밀양-2018-00096,20180209,055-359-7186,경상남도 밀양시,http://www.animal.go.kr/files/shelter/2018/02/...,종료(입양),F,귀가 크고 쫑긋하고 발톱이 길고 날카로웠으며 피부병이 있었음,6(Kg)
3,2016(년생),충청북도 청주시 흥덕구 강내면 서부로 411-55 (강내면),반려동물보호센터,043-201-2298,박종관,힌색,443571201800122,http://www.animal.go.kr/files/shelter/2018/01/...,20180131,상당구 가덕면 수곡리,...,20180219,충북-청주-2018-00140,20180209,043-201-2296,충청북도 청주시,http://www.animal.go.kr/files/shelter/2018/01/...,종료(안락사),M,털이 엉켜있음,3(Kg)
4,2018(년생),충청북도 청주시 흥덕구 강내면 서부로 411-55 (강내면),반려동물보호센터,043-201-2298,박종관,흰색,443571201800121,http://www.animal.go.kr/files/shelter/2018/01/...,20180131,죽림동 137-1,...,20180219,충북-청주-2018-00139,20180209,043-201-2296,충청북도 청주시,http://www.animal.go.kr/files/shelter/2018/01/...,종료(자연사),M,진도개 새끼 6개월 추정 온순하다,1(Kg)
5,2018(년생),충청북도 청주시 흥덕구 강내면 서부로 411-55 (강내면),반려동물보호센터,043-201-2298,박종관,흰색,443571201800118,http://www.animal.go.kr/files/shelter/2018/01/...,20180131,남이면 갈원리 38번지 산,...,20180219,충북-청주-2018-00137,20180209,043-201-2296,충청북도 청주시,http://www.animal.go.kr/files/shelter/2018/01/...,보호중,F,임신한 상태로 유기되어 신고자분 하우스에서 새끼 2마리 놓음,3(Kg)
6,2018(년생),충청북도 청주시 흥덕구 강내면 서부로 411-55 (강내면),반려동물보호센터,043-201-2298,박종관,갈색 믹스견,443571201800117,http://www.animal.go.kr/files/shelter/2018/01/...,20180131,사직동 652-1 1층,...,20180219,충북-청주-2018-00136,20180209,043-201-2296,충청북도 청주시,http://www.animal.go.kr/files/shelter/2018/01/...,종료(반환),M,온순하고 사람을 잘따름,3(Kg)
7,2018(년생),충청북도 청주시 흥덕구 강내면 서부로 411-55 (강내면),반려동물보호센터,043-201-2298,박종관,검정 갈색,443571201800116,http://www.animal.go.kr/files/shelter/2018/01/...,20180131,봉명동 운천시장안 골목대장,...,20180219,충북-청주-2018-00135,20180209,043-201-2296,충청북도 청주시,http://www.animal.go.kr/files/shelter/2018/01/...,종료(반환),M,사람을 잘따른다,10(Kg)
8,2015(년생),경상북도 김천시 조마로 2128 (양천동),김천시 동물보호센터,010-3477-7771,김천시 심은영,갈색,447506201800033,http://www.animal.go.kr/files/shelter/2018/02/...,20180131,율곡동 로제니아호텔부근,...,20180219,경북-김천-2018-00034,20180206,054-421-2520,경상북도 김천시,http://www.animal.go.kr/files/shelter/2018/02/...,종료(안락사),F,온순하며 임신 중으로 보임.,2.03(Kg)
9,2018(년생),경상북도 김천시 조마로 2128 (양천동),김천시 동물보호센터,010-3477-7771,김천시 심은영,갈색,447506201800032,http://www.animal.go.kr/files/shelter/2018/02/...,20180131,김천폐차장,...,20180219,경북-김천-2018-00033,20180206,054-421-2520,경상북도 김천시,http://www.animal.go.kr/files/shelter/2018/02/...,종료(입양),F,"수컷 4마리, 암컷 2마리",1.01(Kg)


위와 같이 데이터프레임 형식으로 데이터를 받아올 수 있습니다.
체중이나 출생연도 값이 숫자나 연도로 지정된 포맷이 아니기 때문에 분석에 활용하려면 전처리가 필요합니다.